In [40]:
import os
from dotenv import load_dotenv
import pandas as pd
import requests
import time
from itertools import islice
import tqdm
load_dotenv()

API_KEY = os.environ.get('SHIV_LASTFM_API_KEY')
BASE_URL = "http://ws.audioscrobbler.com/2.0/"

#https://www.last.fm/api/accounts

# No specific rate limits listed on the website

#### Table 1 - LasFM Tags / Genres

In [37]:
tracks_per_tag = 100
max_tags = 10


params = {
    "method": "tag.getTopTags",
    "api_key": API_KEY,
    "format": "json",
}
top_tags_r = requests.get(BASE_URL, params=params)
top_tags_data = top_tags_r.json()
tags_df = pd.DataFrame(top_tags_data['toptags']['tag'][:max_tags])
tags_df.head()

tags_df.to_csv('data/lastfm/tags_df.csv', index=False)
tags_df

,name,count,reach
0,rock,4052597,401842
1,electronic,2478519,260966
2,seen live,2183712,82503
3,alternative,2125342,266526
4,pop,2063007,232762
5,indie,2059292,259551
6,female vocalists,1630676,169026
7,metal,1297121,158472
8,alternative rock,1219296,169652
9,jazz,1189076,149478


### Table 2 - Tag Ranks

In [27]:
tag_names = tags_df['name'].tolist()
all_tracks = []

for tag in tag_names:   
    tag_tracks = []
    page = 1

    while len(tag_tracks) < tracks_per_tag: 
        params = {
            "method": "tag.getTopTracks",
            "tag": tag,
            "api_key": API_KEY,
            "format": "json",
            "limit": 1000,
            "page": page
        }
        response = requests.get(BASE_URL, params=params)
        tracks_json = response.json()
        
        for track in tracks_json['tracks']['track']:
            track['tag'] = tag
            track['rank'] = ((page-1)*50)+int(track['@attr']['rank'])
            track['artist_name'] = track['artist']['name']
            track['artist_mbid'] = track['artist']['mbid']
        
        tag_tracks.extend(tracks_json['tracks']['track'])
        page += 1
        
    for track in tag_tracks:
        track['tag'] = tag 
    
    all_tracks.extend(tag_tracks)
    print('Tag: ', tag, 'completed')

Tag:  rock completed
Tag:  electronic completed
Tag:  seen live completed
Tag:  alternative completed
Tag:  pop completed
Tag:  indie completed
Tag:  female vocalists completed
Tag:  metal completed
Tag:  alternative rock completed
Tag:  jazz completed


In [45]:
pd.DataFrame(all_tracks)

,name,duration,mbid,url,streamable,artist,image,@attr,tag,rank,artist_name,artist_mbid
0,Smells Like Teen Spirit,301,0ebe2d92-a11d-4b2b-9922-806383074ed7,https://www.last.fm/music/Nirvana/_/Smells+Lik...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Nirvana', 'mbid': '9282c8b4-ca0b-4c6...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '1'},rock,1,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6
1,Mr. Brightside,224,37d516ab-d61f-4bcb-9316-7a0b3eb845a8,https://www.last.fm/music/The+Killers/_/Mr.+Br...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'The Killers', 'mbid': '95e1ead9-4d31...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '2'},rock,2,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b
2,Creep,239,d11fcceb-dfc5-4d19-b45d-f4e8f6d3eaa6,https://www.last.fm/music/Radiohead/_/Creep,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Radiohead', 'mbid': 'a74b1b7f-71a5-4...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '3'},rock,3,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711
3,Come as You Are,208,e05035a3-14ac-4f88-a160-0a144530004e,https://www.last.fm/music/Nirvana/_/Come+as+Yo...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Nirvana', 'mbid': '9282c8b4-ca0b-4c6...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '4'},rock,4,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6
4,Feel Good Inc.,221,5660558a-bfa7-416f-99d9-a34ca0a34515,https://www.last.fm/music/Gorillaz/_/Feel+Good...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Gorillaz', 'mbid': 'e21857d5-3256-45...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '5'},rock,5,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Summertime,695,a8661daf-63c6-4ef1-a746-ce162abb11ef,https://www.last.fm/music/John+Coltrane/_/Summ...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'John Coltrane', 'mbid': 'b625448e-bf...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '996'},jazz,996,John Coltrane,b625448e-bf4a-41c3-a421-72ad46cdb831
9996,Why Can't We Live Together,327,f8d77c6f-40dd-48cd-91ea-d06dd9e3e9f1,https://www.last.fm/music/Sade/_/Why+Can%27t+W...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Sade', 'mbid': '67930b3e-e00b-469f-8...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '997'},jazz,997,Sade,67930b3e-e00b-469f-8c74-fd69f20522ec
9997,As the Stars Fall,0,81e4a6c6-bc5f-4b62-8ab8-406d5730bec3,https://www.last.fm/music/The+Cinematic+Orches...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'The Cinematic Orchestra', 'mbid': '7...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '998'},jazz,998,The Cinematic Orchestra,7c158ea8-c0aa-410e-bdc1-20bba9759577
9998,Temptation,217,e015a013-5083-4701-9331-4afbdf9f9ceb,https://www.last.fm/music/Diana+Krall/_/Tempta...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Diana Krall', 'mbid': '67d2cb7a-9ddb...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '999'},jazz,999,Diana Krall,67d2cb7a-9ddb-4a7f-82bf-5a2d1a038e98


In [38]:
tag_rank_df = pd.DataFrame(all_tracks)[['name','mbid','artist_name','artist_mbid','tag','rank']].rename({'mbid':'track_mbid','name':'track_name'},axis=1)
tag_rank_df.to_csv('data/lastfm/tag_rank_df.csv')
tag_rank_df


,track_name,track_mbid,artist_name,artist_mbid,tag,rank
0,Smells Like Teen Spirit,0ebe2d92-a11d-4b2b-9922-806383074ed7,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,rock,1
1,Mr. Brightside,37d516ab-d61f-4bcb-9316-7a0b3eb845a8,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,rock,2
2,Creep,d11fcceb-dfc5-4d19-b45d-f4e8f6d3eaa6,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711,rock,3
3,Come as You Are,e05035a3-14ac-4f88-a160-0a144530004e,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,rock,4
4,Feel Good Inc.,5660558a-bfa7-416f-99d9-a34ca0a34515,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,rock,5
...,...,...,...,...,...,...
9995,Summertime,a8661daf-63c6-4ef1-a746-ce162abb11ef,John Coltrane,b625448e-bf4a-41c3-a421-72ad46cdb831,jazz,996
9996,Why Can't We Live Together,f8d77c6f-40dd-48cd-91ea-d06dd9e3e9f1,Sade,67930b3e-e00b-469f-8c74-fd69f20522ec,jazz,997
9997,As the Stars Fall,81e4a6c6-bc5f-4b62-8ab8-406d5730bec3,The Cinematic Orchestra,7c158ea8-c0aa-410e-bdc1-20bba9759577,jazz,998
9998,Temptation,e015a013-5083-4701-9331-4afbdf9f9ceb,Diana Krall,67d2cb7a-9ddb-4a7f-82bf-5a2d1a038e98,jazz,999


### Table 3 - Track Info

In [32]:

mbids = tag_rank_df['track_mbid'].unique()
tracks = []
for mbid in tqdm.tqdm(mbids):
    try:
        params = {
            "method": "track.getInfo",
            "api_key": API_KEY,
            "mbid": mbid,
            "format": "json"
        }
        response = requests.get(BASE_URL, params=params)
        tracks.append(response.json())
    except:
        print('Error: ', mbid)

    
tracks_df = pd.DataFrame([t['track'] for t in tracks])


  0%|          | 0/4508 [00:00<?, ?it/s]

0ebe2d92-a11d-4b2b-9922-806383074ed7


In [39]:
tracks_df.to_csv('data/lastfm/tracks_df.csv', index=False)
tracks_df

,name,mbid,url,duration,streamable,listeners,playcount,artist,album,toptags,wiki
0,Knocked Up,6956c27e-3c15-494f-bff9-af2241f2506c,https://www.last.fm/music/Kings+of+Leon/_/Knoc...,430000,"{'#text': '0', 'fulltrack': '0'}",491594,3039772,"{'name': 'Kings of Leon', 'mbid': '6ffb8ea9-23...","{'artist': 'Kings of Leon', 'title': 'Because ...","{'tag': [{'name': 'rock', 'url': 'https://www....","{'published': '13 Jun 2008, 20:12', 'summary':..."
1,Need You Tonight,aa12dc94-77b1-4e9b-990f-b2bae959f5e0,https://www.last.fm/music/INXS/_/Need+You+Tonight,189000,"{'#text': '0', 'fulltrack': '0'}",802799,4589591,"{'name': 'INXS', 'mbid': '481bf5f9-2e7c-4c44-b...","{'artist': 'INXS', 'title': 'Kick', 'mbid': '1...","{'tag': [{'name': '80s', 'url': 'https://www.l...","{'published': '31 Jan 2009, 17:14', 'summary':..."
2,Fitter Happier,5838f978-0822-4e28-874f-e1511324ec3a,https://www.last.fm/music/Radiohead/_/Fitter+H...,117000,"{'#text': '0', 'fulltrack': '0'}",1070145,6165992,"{'name': 'Radiohead', 'mbid': 'a74b1b7f-71a5-4...","{'artist': 'Radiohead', 'title': 'OK Computer'...","{'tag': [{'name': 'alternative', 'url': 'https...","{'published': '31 Mar 2009, 19:12', 'summary':..."
3,Mercy,e528371a-0544-4328-a51e-8a2427671fcc,https://www.last.fm/music/Duffy/_/Mercy,220000,"{'#text': '0', 'fulltrack': '0'}",980004,5628145,"{'name': 'Duffy', 'mbid': 'da4f7583-bb31-49d2-...","{'artist': 'Duffy', 'title': 'Deluxe EP', 'mbi...","{'tag': [{'name': 'soul', 'url': 'https://www....","{'published': '08 Nov 2008, 11:10', 'summary':..."
4,Blue Ridge Mountains,310fa155-f188-46bb-b39a-f5ab8ac048df,https://www.last.fm/music/Fleet+Foxes/_/Blue+R...,200000,"{'#text': '0', 'fulltrack': '0'}",701071,5366356,"{'name': 'Fleet Foxes', 'mbid': 'fa97dd36-1b82...","{'artist': 'Fleet Foxes', 'title': 'Fleet Foxe...","{'tag': [{'name': 'folk', 'url': 'https://www....",NaN
...,...,...,...,...,...,...,...,...,...,...,...
4501,This Fire,da22fa3b-ee21-4326-adda-cfb2dabaaecf,https://www.last.fm/music/Franz+Ferdinand/_/Th...,255000,"{'#text': '0', 'fulltrack': '0'}",853912,4660231,"{'name': 'Franz Ferdinand', 'mbid': 'aa7a2827-...","{'artist': 'Franz Ferdinand', 'title': 'Franz ...","{'tag': [{'name': 'indie', 'url': 'https://www...","{'published': '29 Jan 2009, 13:52', 'summary':..."
4502,Forgotten,241e711d-e836-433c-b4bd-b2340f199f7b,https://www.last.fm/music/Linkin+Park/_/Forgotten,194000,"{'#text': '0', 'fulltrack': '0'}",759399,5715870,"{'name': 'Linkin Park', 'mbid': 'f59c5520-5f46...","{'artist': 'Linkin Park', 'title': 'Hybrid The...","{'tag': [{'name': 'Nu Metal', 'url': 'https://...","{'published': '26 Jun 2024, 10:18', 'summary':..."
4503,Shimmy,668b2928-dbc8-401a-9b2d-6fe546dcd441,https://www.last.fm/music/System+of+a+Down/_/S...,111000,"{'#text': '0', 'fulltrack': '0'}",765557,4783874,"{'name': 'System of a Down', 'mbid': 'cc0b7089...","{'artist': 'System of a Down', 'title': 'Great...","{'tag': [{'name': 'alternative metal', 'url': ...",NaN
4504,Mermaid,42640092-938d-49b7-a5ff-2a54f3a33175,https://www.last.fm/music/Sade/_/Mermaid,264000,"{'#text': '0', 'fulltrack': '0'}",154552,756496,"{'name': 'Sade', 'mbid': '67930b3e-e00b-469f-8...","{'artist': 'Sade', 'title': 'Love Deluxe', 'mb...","{'tag': [{'name': 'soul', 'url': 'https://www....","{'published': '02 Aug 2024, 17:30', 'summary':..."
